# 01 — Data Cleaning
Parse timestamps, handle missing values, convert units, pivot weather wide, merge, add time features.

In [1]:
import pandas as pd
import numpy as np

# Load raw data
energy = pd.read_csv('../energy_dataset.csv', parse_dates=['time'])
weather = pd.read_csv('../weather_features.csv', parse_dates=['dt_iso'])

print(f"Energy: {energy.shape}")
print(f"Weather: {weather.shape}")

Energy: (35064, 29)
Weather: (178396, 17)


In [2]:
# Check missing values in energy
missing = energy.isnull().sum()
print(missing[missing > 0])

generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar                                  18
generation waste                              

In [3]:
# Drop columns that are entirely/mostly null or not needed
drop_cols = [
    'generation fossil coal-derived gas',
    'generation fossil oil shale',
    'generation fossil peat',
    'generation geothermal',
    'generation hydro pumped storage aggregated',
    'generation marine',
    'generation wind offshore',
    'forecast wind offshore eday ahead',
]
energy = energy.drop(columns=[c for c in drop_cols if c in energy.columns])

# Interpolate small gaps in remaining numeric columns
numeric_cols = energy.select_dtypes(include=[np.number]).columns
energy[numeric_cols] = energy[numeric_cols].interpolate(method='linear', limit=3)

# Drop rows still having nulls in key columns
key_cols = ['total load actual', 'price actual', 'generation wind onshore']
energy = energy.dropna(subset=key_cols)

print(f"Energy after cleaning: {energy.shape}")
print(f"Remaining nulls: {energy.isnull().sum().sum()}")

Energy after cleaning: (35056, 21)
Remaining nulls: 0


In [4]:
# --- Weather cleaning ---
# Keep only numeric weather features
weather_numeric = ['temp', 'temp_min', 'temp_max', 'pressure', 'humidity',
                    'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h', 'clouds_all']

# Fix leading space in city names (raw data has ' Barcelona')
weather['city_name'] = weather['city_name'].str.strip()

# Convert temperature Kelvin -> Celsius
for col in ['temp', 'temp_min', 'temp_max']:
    weather[col] = weather[col] - 273.15

# Fix pressure outliers: replace >1100 hPa and ==0 with NaN, then interpolate per city
bad_pressure = (weather['pressure'] > 1100) | (weather['pressure'] == 0)
print(f"Pressure outliers: {bad_pressure.sum()} (max was {weather['pressure'].max():.0f})")
weather.loc[bad_pressure, 'pressure'] = np.nan
weather['pressure'] = weather.groupby('city_name')['pressure'].transform(
    lambda x: x.interpolate(method='linear')
)

# Fix wind speed outliers: replace >30 m/s with NaN, then interpolate per city
bad_wind = weather['wind_speed'] > 30
print(f"Wind speed outliers: {bad_wind.sum()} (max was {weather['wind_speed'].max():.0f})")
weather.loc[bad_wind, 'wind_speed'] = np.nan
weather['wind_speed'] = weather.groupby('city_name')['wind_speed'].transform(
    lambda x: x.interpolate(method='linear')
)

# Drop duplicate (city, timestamp) rows — keep first
weather = weather.drop_duplicates(subset=['dt_iso', 'city_name'], keep='first')

print(f"Weather after cleaning: {weather.shape}")
print(f"Cities: {weather['city_name'].unique()}")

Pressure outliers: 47 (max was 1008371)
Wind speed outliers: 8 (max was 133)
Weather after cleaning: (175320, 17)
Cities: ['Valencia' 'Madrid' 'Bilbao' 'Barcelona' 'Seville']


In [5]:
# Pivot weather wide: one row per timestamp, columns like temp_madrid, wind_speed_bilbao, etc.
weather_pivot = weather.pivot_table(
    index='dt_iso',
    columns='city_name',
    values=weather_numeric,
    aggfunc='first'
)

# Flatten multi-level columns
weather_pivot.columns = [
    f"{feat}_{city.lower().replace(' ', '_')}"
    for feat, city in weather_pivot.columns
]
weather_pivot = weather_pivot.reset_index()

print(f"Weather pivoted: {weather_pivot.shape}")
print(weather_pivot.columns.tolist()[:10], '...')

Weather pivoted: (35064, 56)
['dt_iso', 'clouds_all_barcelona', 'clouds_all_bilbao', 'clouds_all_madrid', 'clouds_all_seville', 'clouds_all_valencia', 'humidity_barcelona', 'humidity_bilbao', 'humidity_madrid', 'humidity_seville'] ...


In [6]:
# Merge energy + weather on timestamp
# Normalize timestamps to UTC for merge
energy['time'] = pd.to_datetime(energy['time'], utc=True)
weather_pivot['dt_iso'] = pd.to_datetime(weather_pivot['dt_iso'], utc=True)

df = energy.merge(weather_pivot, left_on='time', right_on='dt_iso', how='inner')
df = df.drop(columns=['dt_iso'])
df = df.sort_values('time').reset_index(drop=True)

print(f"Merged: {df.shape}")

Merged: (35056, 76)


In [7]:
# Add time features
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.dayofweek
df['month'] = df['time'].dt.month
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

print(f"Final shape: {df.shape}")
print(f"Date range: {df['time'].min()} to {df['time'].max()}")
df.head()

Final shape: (35056, 80)
Date range: 2014-12-31 23:00:00+00:00 to 2018-12-31 22:00:00+00:00


,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,...,wind_deg_valencia,wind_speed_barcelona,wind_speed_bilbao,wind_speed_madrid,wind_speed_seville,wind_speed_valencia,hour,day_of_week,month,is_weekend
0,2014-12-31 23:00:00+00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,...,62,7.0,0.0,1.0,1.0,1.0,23,2,12,0
1,2015-01-01 00:00:00+00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,...,62,7.0,0.0,1.0,1.0,1.0,0,3,1,0
2,2015-01-01 01:00:00+00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,...,23,7.0,1.0,1.0,3.0,0.0,1,3,1,0
3,2015-01-01 02:00:00+00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,...,23,7.0,1.0,1.0,3.0,0.0,2,3,1,0
4,2015-01-01 03:00:00+00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,23,7.0,1.0,1.0,3.0,0.0,3,3,1,0


In [8]:
# Save cleaned data
df.to_parquet('../cleaned_data.parquet', index=False)
print('Saved cleaned_data.parquet')
print(f"Columns ({len(df.columns)}): {df.columns.tolist()}")

Saved cleaned_data.parquet
Columns (80): ['time', 'generation biomass', 'generation fossil brown coal/lignite', 'generation fossil gas', 'generation fossil hard coal', 'generation fossil oil', 'generation hydro pumped storage consumption', 'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation nuclear', 'generation other', 'generation other renewable', 'generation solar', 'generation waste', 'generation wind onshore', 'forecast solar day ahead', 'forecast wind onshore day ahead', 'total load forecast', 'total load actual', 'price day ahead', 'price actual', 'clouds_all_barcelona', 'clouds_all_bilbao', 'clouds_all_madrid', 'clouds_all_seville', 'clouds_all_valencia', 'humidity_barcelona', 'humidity_bilbao', 'humidity_madrid', 'humidity_seville', 'humidity_valencia', 'pressure_barcelona', 'pressure_bilbao', 'pressure_madrid', 'pressure_seville', 'pressure_valencia', 'rain_1h_barcelona', 'rain_1h_bilbao', 'rain_1h_madrid', 'rain_1h_seville', 'rain_1h